In [180]:
# Considerar para o dataset apenas contas mais generalizantes (1.02.01 -> no máximo três números!)
# Escolher apenas o Ticker 3 quando há várias ações
# Tratar números infinitos e NaN. Melhor colocá-los como 0.

# Alterar os dados do dataset. Ao invés dos valores brutos, colocar a variação das contas por empresa ao longo dos semestres.

<a href='https://www.idinheiro.com.br/cnpj-empresas-listadas-b3/'> CNPJ e Ticker </a>

<h1 style='font-size:40px'> Stock Price Variation Predictor</h1>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            The present project aims to create a stock price variation predictor using the  BOVESPA corporations' fundamentals. 
        </li>
        <li> 
            The model must estimate the given quarter's 80th percentile variation using the financial statements from the previous quarter.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Building the dataset</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Unlike most of the ML projects out there, in this one we'll face the challenge of making the dataset with which our models will carry out their predictions.
        </li>
        <li> 
            We are going to use wget in order to download the files and extract the financial statements.
        </li>
    </ul>
</div>

<a href='https://www.youtube.com/watch?v=LCMgVO1BOVQ&t=659s'> Link Tutorial</a>

In [2]:
import wget
import pandas as pd
from datetime import date
from zipfile import ZipFile
from pathlib import Path
import os
import numpy as np

In [9]:
import wget
import pandas as pd
from datetime import date
from zipfile import ZipFile
from pathlib import Path
import os

# The data source to the financial statements is going to be the URL below.
data_source = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'

# Listing the documents from 2015 until last year.
current_year = date.today().year
zip_files = [f'itr_cia_aberta_{year}.zip' for year in range(2015, current_year)]

# Downloading the files and extracting their contents.
for file in zip_files:
    wget.download(data_source + file)
    with ZipFile(file, 'r') as zip_file:
        zip_file.extractall('Statements')
    # After the extraction is completed, we are able to delete the zip file.
    os.remove(file)

# The zip files contained two kinds of reports, the ones with '_con_' and others with '_ind_' in their names.
# Since we are going to use only the first sort of document, we'll remove the '_ind_' ones.

# Also, there were two kinds of DMPL statements, the '_MD_' and the '_MI_'. I ch
for file in Path('Statements').iterdir():
    if '_ind_' in file.name or '_MI_' in file.name:
        os.remove(file)

100% [....................................................] 29984400 / 29984400

In [47]:
with open('.gitignore', 'w') as git:
    for file in os.listdir('Statements'):
        git.write(f'Statements/{file}\n')
            
git.close()

In [38]:
from re import search
# This function creates a DataFrame with all the reports of a given kind of statement.
def unite_reports(type_statement):
    df = pd.DataFrame()
    #financial_statements = '(BPA|BPP|DFC|DMPL|DRA|DRE|DVA)'#['BPA', 'BPP', 'DFC', 'DMPL', 'DRA', 'DRE', 'DVA']
    for file in os.listdir('Statements'):
        if search(type_statement, file):
            #print(1)
            financial_statement_df = pd.read_csv(f'Statements/{file}', encoding='ISO-8859-1', sep=';', parse_dates=['DT_FIM_EXERC'])
            financial_statement_df = financial_statement_df.query('''ORDEM_EXERC=='ÚLTIMO' & ST_CONTA_FIXA=='S' & VERSAO==1 ''')
            df = pd.concat([df, financial_statement_df])
    pivot = pd.pivot_table(df, values='VL_CONTA', columns='CD_CONTA', index=['CNPJ_CIA', 'DT_FIM_EXERC'])
    # Retrieving a DataFrame with the accounts percentage change. Removing the lines that based the pct change comparison.
    return pivot.groupby('CNPJ_CIA').pct_change().dropna(how='all').fillna(method='bfill')

# This method converts the DF's level 1 data type to PeriodIndex.
def period_index(df):
    quarter_index = pd.PeriodIndex(df.xs(key=slice(None), level=0).index, freq='Q')
    return df.set_index([df.xs(key=slice(None), level=1).index , quarter_index])

# The 'dataset' function creates the final dataset by appending the results of 'unite_reports' for every statement kind available.
def dataset():
    df = unite_reports('BPA')
    for report in ['BPP', 'DFC', 'DMPL', 'DRA', 'DRE', 'DVA']:
        df = df.join(unite_reports(report))
    # Here, we are filtering out corporations with less than 12 quarters of available data.    
    sizes = df.groupby('CNPJ_CIA').size()
    df.loc(axis=0)[sizes[sizes>12].index,:]
    return period_index(df)

In [39]:
df = dataset()

In [246]:
acoes[acoes['CNPJ']=='00.001.180/0001-26']

,Código(s),CNPJ
49,ELET3ELET5ELET6,00.001.180/0001-26


In [259]:
acoes.to_csv('CNPJ-Ticker.csv')

In [57]:
df.replace({np.inf:0, -np.inf:0 ,np.nan:0})

CD_CONTA                                1      1.01   1.01.01   1.01.02  \
CNPJ_CIA           DT_FIM_EXERC                                           
00.001.180/0001-26 2015Q2       -0.010043 -0.096479 -0.252623  0.508151   
                   2015Q3        0.011767 -0.022021  0.376352 -0.012629   
                   2016Q1       -0.023885 -0.059246  0.072131 -0.214369   
                   2016Q2        0.161198 -0.047628 -0.144691  0.147203   
                   2016Q3        0.011992 -0.021155 -0.028056 -0.075112   
...                                   ...       ...       ...       ...   
97.837.181/0001-47 2020Q2        0.048010  0.075440  0.202509  0.000000   
                   2020Q3        0.021755 -0.000578 -0.024076  0.000000   
                   2021Q1        0.001332  0.005451 -0.190893  0.000000   
                   2021Q2        0.059403  0.060232  0.050982  0.000000   
                   2021Q3        0.048843  0.083526  0.060078  0.000000   

CD_CONTA                         1.01.02.01  1.01.02.01.01  1.01.02.01.02  \
CNPJ_CIA           DT_FIM_EXERC                                             
00.001.180/0001-26 2015Q2               0.0      -0.370693            0.0   
                   2015Q3               0.0      -0.370693            0.0   
                   2016Q1               0.0      -0.370693            0.0   
                   2016Q2               0.0      -0.370693            0.0   
                   2016Q3               0.0      -0.370693            0.0   
...                                     ...            ...            ...   
97.837.181/0001-47 2020Q2               0.0       0.000000            0.0   
                   2020Q3               0.0       0.000000            0.0   
                   2021Q1               0.0       0.000000            0.0   
                   2021Q2               0.0       0.000000            0.0   
                   2021Q3               0.0       0.000000            0.0   

CD_CONTA                         1.01.02.02  1.01.02.02.01  1.01.02.03  \
CNPJ_CIA           DT_FIM_EXERC                                          
00.001.180/0001-26 2015Q2          0.508151       0.508151   -0.445259   
                   2015Q3         -0.012629      -0.012629   -0.445259   
                   2016Q1         -0.214369      -0.214369   -0.445259   
                   2016Q2          0.147203       0.147203   -0.445259   
                   2016Q3         -0.075112      -0.075112   -0.445259   
...                                     ...            ...         ...   
97.837.181/0001-47 2020Q2          0.000000       0.000000    0.000000   
                   2020Q3          0.000000       0.000000    0.000000   
                   2021Q1          0.000000       0.000000    0.000000   
                   2021Q2          0.000000       0.000000    0.000000   
                   2021Q3          0.000000       0.000000    0.000000   

CD_CONTA                          1.01.03  1.01.03.01  1.01.03.02  1.01.03.03  \
CNPJ_CIA           DT_FIM_EXERC                                                 
00.001.180/0001-26 2015Q2       -0.045984   -0.045984    0.000000         0.0   
                   2015Q3        0.132727    0.132727    0.000000         0.0   
                   2016Q1       -0.190099   -0.190099    0.000000         0.0   
                   2016Q2        0.070030    0.070030    0.000000         0.0   
                   2016Q3        0.007639    0.007639    0.000000         0.0   
...                                   ...         ...         ...         ...   
97.837.181/0001-47 2020Q2        0.022738    0.014413    0.344673         0.0   
                   2020Q3        0.164111    0.170923   -0.034639         0.0   
                   2021Q1       -0.001205   -0.026840    0.905955         0.0   
                   2021Q2        0.087364    0.087667    0.081890         0.0   
                   2021Q3        0.111201    0.126161   -0.160540         0.0   

CD_CO

In [191]:
# 'acoes' lê a página que possui os CNPJ's e os Tickers das empresas.
acoes = pd.read_html('https://www.idinheiro.com.br/cnpj-empresas-listadas-b3/')[0].iloc[:, [1,2]]

In [204]:
# É possível resgatar preços de ações desde 2011
import pandas_datareader as web
web.DataReader(f'PETR3.SA', 'yahoo', start='01/01/2011')

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2011-01-03,30.870001,30.120001,30.719999,30.299999,5064700.0,17.491817
2011-01-04,30.400000,29.540001,30.299999,30.059999,10066400.0,17.353264
2011-01-05,30.670000,29.709999,29.799999,30.629999,5502100.0,17.682322
2011-01-06,30.850000,30.309999,30.549999,30.450001,6889400.0,17.578415
2011-01-07,30.510000,29.910000,30.389999,30.000000,5344500.0,17.318630
...,...,...,...,...,...,...
2022-06-06,34.110001,33.380001,33.889999,33.740002,9285200.0,33.740002
2022-06-07,34.630001,33.549999,33.549999,33.860001,10982800.0,33.860001
2022-06-08,34.230000,33.560001,33.740002,33.730000,7258500.0,33.730000


In [190]:
acoes = pd.read_html('https://www.idinheiro.com.br/cnpj-empresas-listadas-b3/')[0].iloc[:, [1,2]]

<h3 style='font-size:30px;font-style:italic'> itr_cia_aberta</h3>

In [35]:
pd.read_csv('Statements/itr_cia_aberta_BPA_ind_2011.csv', encoding='ISO-8859-1', sep=';')['DS_CONTA'].head(3)

0         Ativo Total
1         Ativo Total
2    Ativo Circulante
Name: DS_CONTA, dtype: object

In [28]:
pd.read_csv('Statements/itr_cia_aberta_BPA_ind_2012.csv', encoding='ISO-8859-1', sep=';').head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.000.000/0001-91,2012-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2011-12-31,1,Ativo Total,890352257.0,S
1,00.000.000/0001-91,2012-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2012-03-31,1,Ativo Total,913866693.0,S
2,00.000.000/0001-91,2012-03-31,1,BCO BRASIL S.A.,1023,DF Individual - Balanço Patrimonial Ativo,REAL,MIL,PENÚLTIMO,2011-12-31,1.01,Ativo Circulante,518716710.0,S


<h3 style='font-size:30px;font-style:italic'> BPA</h3>

In [83]:
x = pd.read_csv('Statements/itr_cia_aberta_BPA_con_2015.csv', encoding='ISO-8859-1', sep=';',parse_dates=['DT_FIM_EXERC'])
x[(x['CD_CONTA']=='1.02.01.10.02') & (x['ORDEM_EXERC']=='ÚLTIMO') & (x['CNPJ_CIA']=='00.001.180/0001-26')]

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
1,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2015-03-31,1,Ativo Total,152427122.0,S
167,00.001.180/0001-26,2015-06-30,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2015-06-30,1,Ativo Total,150896328.0,S
333,00.001.180/0001-26,2015-09-30,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,ÚLTIMO,2015-09-30,1,Ativo Total,152671980.0,S


<h3 style='font-size:30px;font-style:italic'> BPP</h3>

In [64]:
x = pd.read_csv('Statements/itr_cia_aberta_BPP_con_2015.csv', encoding='ISO-8859-1', sep=';')
x

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2014-12-31,2,Passivo Total,144631697.0,S
1,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2015-03-31,2,Passivo Total,152427122.0,S
2,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2014-12-31,2.01,Passivo Circulante,19284008.0,S
3,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2015-03-31,2.01,Passivo Circulante,22083808.0,S
4,00.001.180/0001-26,2015-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2014-12-31,2.01.01,Obrigações Sociais e Trabalhistas,0.0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216667,97.837.181/0001-47,2015-09-30,1,DURATEX S.A.,21091,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2015-09-30,2.03.07,Ajustes Acumulados de Conversão,0.0,S
216668,97.837.181/0001-47,2015-09-30,1,DURATEX S.A.,21091,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2014-12-31,2.03.08,Outros Resultados Abrangentes,0.0,S
216669,97.837.181/0001-47,2015-09-30,1,DURATEX S.A.,21091,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,ÚLTIMO,2015-09-30,2.03.08,Outros Resultados Abrangentes,0.0,S
216670,97.837.181/0001-47,2015-09-30,1,DURATEX S.A.,21091,DF Consolidado - Balanço Patrimonial Passivo,REAL,MIL,PENÚLTIMO,2014-12-31,2.03.09,Participação dos Acionistas Não Controladores,65801.0,S


<h3 style='font-size:30px;font-style:italic'> DFC_MD</h3>

In [31]:
pd.read_csv('Statements/itr_cia_aberta_DFC_MD_con_2011.csv', encoding='ISO-8859-1', sep=';').head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.070.698/0001-11,2011-03-31,1,CIA ENERGETICA DE BRASILIA,14451,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,6.01,Caixa Líquido Atividades Operacionais,14679.0,S
1,00.070.698/0001-11,2011-03-31,1,CIA ENERGETICA DE BRASILIA,14451,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,6.01,Caixa Líquido Atividades Operacionais,18238.0,S
2,00.070.698/0001-11,2011-03-31,1,CIA ENERGETICA DE BRASILIA,14451,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,6.01.01,Recebimento de Consumidores,472870.0,N


<h3 style='font-size:30px;font-style:italic'> DFC_MI</h3>

In [39]:
pd.read_csv('Statements/itr_cia_aberta_DFC_MI_con_2011.csv', encoding='ISO-8859-1', sep=';').head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,6.01,Caixa Líquido Atividades Operacionais,1619666.0,S
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,6.01,Caixa Líquido Atividades Operacionais,1189117.0,S
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Fluxo de Caix...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,6.01.01,Caixa Gerado nas Operações,2407485.0,S


<h3 style='font-size:30px;font-style:italic'> DMPL</h3>

In [52]:
pd.read_csv('Statements/itr_cia_aberta_DMPL_con_2015.csv', encoding='ISO-8859-1', sep=';')['VL_CONTA'].max()

442198000.0

<h3 style='font-size:30px;font-style:italic'> DRA</h3>

In [41]:
pd.read_csv('Statements/itr_cia_aberta_DRA_con_2011.csv', encoding='ISO-8859-1', sep=';').head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Resultado Abr...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,4.01,Lucro Líquido Consolidado do Período,0.0,S
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Resultado Abr...,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,4.01,Lucro Líquido Consolidado do Período,0.0,S
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Resultado Abr...,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,4.02,Outros Resultados Abrangentes,0.0,S


<h3 style='font-size:30px;font-style:italic'> DRE</h3>

In [42]:
pd.read_csv('Statements/itr_cia_aberta_DRE_con_2011.csv', encoding='ISO-8859-1', sep=';').head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.01,Receita de Venda de Bens e/ou Serviços,6159172.0,S
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,3.01,Receita de Venda de Bens e/ou Serviços,8551982.0,S
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração do Resultado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,3.02,Custo dos Bens e/ou Serviços Vendidos,-960060.0,S


<h3 style='font-size:30px;font-style:italic'> DVA</h3>

In [79]:
#f = lambda x: x[x['ORDEM_EXERC']=='ÚLTIMO']
pd.read_csv('Statements/itr_cia_aberta_DVA_con_2011.csv', encoding='ISO-8859-1', sep=';')#.apply(f, axis=1)#.head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
0,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,7.01,Receitas,6354944.0,S
1,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,7.01,Receitas,8753827.0,S
2,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,7.01.01,"Vendas de Mercadorias, Produtos e Serviços",6156290.0,S
3,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2011-01-01,2011-03-31,7.01.01,"Vendas de Mercadorias, Produtos e Serviços",8551982.0,S
4,00.001.180/0001-26,2011-03-31,1,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-03-31,7.01.02,Outras Receitas,198654.0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89505,97.837.181/0001-47,2011-09-30,1,DURATEX S.A.,21091,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,7.08.04.03,Lucros Retidos / Prejuízo do Período,187980.0,S
89506,97.837.181/0001-47,2011-09-30,1,DURATEX S.A.,21091,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-09-30,7.08.04.04,Part. Não Controladores nos Lucros Retidos,283.0,S
89507,97.837.181/0001-47,2011-09-30,1,DURATEX S.A.,21091,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,ÚLTIMO,2011-01-01,2011-09-30,7.08.04.04,Part. Não Controladores nos Lucros Retidos,525.0,S
89508,97.837.181/0001-47,2011-09-30,1,DURATEX S.A.,21091,DF Consolidado - Demonstração de Valor Adicionado,REAL,MIL,PENÚLTIMO,2010-01-01,2010-09-30,7.08.05,Outros,0.0,S
